# CNNs to Decode ECoG Data
Run the first few cells to normalize Local / Colab, then proceed below.

In [ ]:
from pathlib import Path
import os
try:
    # See if we are running on google.colab
    import google.colab
    from google.colab import files
    os.chdir('..')
    if not (Path.cwd() / '.kaggle').is_dir():
        # Configure kaggle
        files.upload()  # Find the kaggle.json file in your ~/.kaggle directory.
        !pip install -q kaggle
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
    if not (Path.cwd() / 'repo').is_dir():
        # Download the workshop repo and change to its directory
        # For now edit the username/password. This requirement will be removed when the repo is made public.
        !git clone https://github.com/SachsLab/IntracranialNeurophysDL.git
        os.chdir('IntracranialNeurophysDL')
    IN_COLAB = True
    # Setup tensorflow 2.0
    !pip install -q tensorflow-gpu==2.0.0-alpha0
    import tensorflow as tf
    # Setup tensorboard callback
    !pip install tensorboardcolab
    from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
    tbc=TensorBoardColab(startup_waiting_time=30)
    tensorboard_callback = TensorBoardColabCallback(tbc)
except ModuleNotFoundError:
    IN_COLAB = False
    import sys
    if Path.cwd().stem == 'notebooks':
        os.chdir(Path.cwd().parent)
    # Make sure the kaggle executable is on the PATH
    os.environ['PATH'] = os.environ['PATH'] + ';' + str(Path(sys.executable).parent / 'Scripts')
    # Clear any logs from previous runs
    if (Path.cwd() / 'logs').is_dir():
        import platform
        if platform.system() == 'Windows':
            !rmdir /S /Q logs
        else:
            !rm -Rf logs
    # Setup tensorboard callback
    import tensorflow as tf
    import datetime
    %load_ext tensorboard.notebook
    log_dir = Path.cwd() / "logs" / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Download and unzip data
datadir = Path.cwd() / 'data' / 'kjm_ecog'
if not (datadir / 'converted').is_dir():
    !kaggle datasets download -d cboulay/kjm-ecog-faces-basic
    print("Finished downloading. Now extracting contents...")
    data_path = Path('kjm-ecog-faces-basic.zip')
    import zipfile
    with zipfile.ZipFile(data_path, 'r') as zip_ref:
        zip_ref.extractall(datadir / 'converted' / 'faces_basic')
    data_path.unlink()
    print("Finished extracting data.")

## Prepare data from one participant
See 02_02 for an explanation.
However, this time we are loading the full-band data at the original sampling rate.
Let's pretend that we don't know anything about what signal features might be important.
Even if we think we know what signal features might be important, sometimes it's nice
to remove our bias and let the machine find the features for us.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from data.utils.fileio import from_neuropype_h5

PTRAIN = 0.8
BATCH_SIZE = 5

# Load data from one participant.
SUB_ID = 'de'
test_file = datadir / 'converted' / 'faces_basic' / (SUB_ID + '_full.h5')
chunks = from_neuropype_h5(test_file)
chunk_names = [_[0] for _ in chunks]
chunk = chunks[chunk_names.index('signals')][1]
ax_types = [_['type'] for _ in chunk['axes']]
instance_axis = chunk['axes'][ax_types.index('instance')]
n_trials = len(instance_axis['data'])
X = chunk['data']
Y = instance_axis['data']['Marker'].values.reshape(-1, 1)
n_trials = X.shape[0]
print(X.shape, Y.shape)

# Convert Y from strings to integers.
classes, y = np.unique(Y, return_inverse=True)
n_classes = len(classes)
n_trials = len(y)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=PTRAIN)
n_train = len(y_train)
n_valid = len(y_valid)
ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
ds_valid = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

def preprocess_fn(x_dat, y_dat):
    x_dat = tf.cast(x_dat, tf.float32)
    y_dat = tf.one_hot(tf.cast(y_dat, tf.uint8), n_classes)
    return x_dat, y_dat
ds_train = ds_train.map(preprocess_fn)
ds_valid = ds_valid.map(preprocess_fn)
ds_train = ds_train.shuffle(int(n_trials * PTRAIN) + 1).batch(BATCH_SIZE, drop_remainder=True).repeat()  # , drop_remainder=True?
ds_valid = ds_valid.batch(BATCH_SIZE).repeat()

## Create our model
As in 02_02, our objective is to decode the stimulus class ('ISI', 'face', or 'house')
from the ECoG data.
Let's start off with a model that someone else developed for similar purposes.

[One for EEG](https://iopscience.iop.org/article/10.1088/1741-2552/aaf3f6)

[CNNs with autoencoders](https://iopscience.iop.org/article/10.1088/1741-2552/aaf13f/pdf)


In [ ]:
# inputs = tf.keras.layers.Input(shape=(X.shape[1],))
# dat_in_model = tf.keras.layers.Dense(3, activation='linear')(inputs)
# outputs = tf.keras.layers.Softmax()(dat_in_model)  # Convert to probability scores, summing to 1
# model = tf.keras.Model(inputs, outputs)
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# model.summary()
